# #Metalens Factory

In [1]:
import numpy as np
%load_ext autoreload
%autoreload 2
import rspie as rs
from matplotlib import pyplot as plt
import cmasher as cm
import os

## ##Circuit maker

Goal: write code that allows creating a PhotonicTools circuit with cylindrical posts of given radius and position on a honeycomb lattice bunded by a circular aperture. All the posts will be of the same height and material. Initially let me just focus on generating the structure itself, so that it looks as intended in RSoft, without even going to simulate it.

for starters let's assume that the metasurface will be simulated in FullWAVE, so that I can start with a template that resembles the usage of that tool

### ###Random posts in random positions

In [64]:
substrate_segment = '''
structure = STRUCT_CHANNEL
extended = 1
begin.x = 0
begin.z = -(2*Offset)
begin.height = 2*ApertureRadius
begin.width = 2*ApertureRadius
begin.delta = 2*SubstrateIndex - background_index
end.x = 0 rel begin segment 1
end.y = 0 rel begin segment 1
end.z = 0
end.height = 2*ApertureRadius
end.width = 2*ApertureRadius
end.delta = 2*SubstrateIndex-background_index
'''
pillar_template = '''
extended = 1
position_taper = TAPER_LINEAR
begin.x = {pillar_x}
begin.y = {pillar_y}
begin.z = 0
begin.width = {pillar_diameter}
begin.height = {pillar_diameter}
begin.delta = PillarIndex-background_index
end.x = {pillar_x}
end.y = {pillar_y}
end.z = PillarHeight
end.width = {pillar_diameter}
end.height = {pillar_diameter}
end.delta = PillarIndex-background_index
'''

In [68]:
extra_config = {
    'ApertureRadius' : 10,
    'PillarHeight': 1.2,
    'background_index': 1,
    'PillarIndex': 1.5,
    'free_space_wavelength': 0.600,
    'SubstrateIndex': 1.5}

config = {'vars'   : {}, 
        'segments' : [],
        'monitors' : [],
        'launch_fields' : {}
        }

config_text = '''
filename = metaopt.ind
ApertureRadius = {ApertureRadius}
Offset = 2*lambda
PillarHeight = {PillarHeight}
PillarIndex = {PillarIndex}
SubstrateIndex = {SubstrateIndex}
Xmax = {ApertureRadius}
Xmin = -{ApertureRadius}
Ymax = {ApertureRadius}
Ymin = -{ApertureRadius}
Zmax = {PillarHeight} + Offset
Zmin = -(2*Offset)
alpha = 0
background_index = {background_index}
boundary_max = Xmax
boundary_max_y = Ymax
boundary_min = Xmin
boundary_min_y = Ymin
cad_aspectratio = 1
delta = index-background_index
dimension = 3
domain_max = Zmax
domain_min = Zmin
eim = 0
fdtd_display_res_auto = DISPLAY_RES_AUTO
fdtd_monitor_time = lambda/4
fdtd_monitor_time_auto = MONITOR_TIME_AUTO
fdtd_pml_cells_enable = 1
fdtd_stop_auto = 1
fdtd_stop_time = 87
fdtd_stop_time_auto = 1
fdtd_time_step = 0.005681818182
fdtd_time_step_auto = 1
fdtd_update_time = 9*lambda/4
fdtd_update_time_auto = DISPLAY_TIME_AUTO
free_space_wavelength = {free_space_wavelength}
width = 1
height = width
index = 1
k0 = (2*pi)/free_space_wavelength
lambda = free_space_wavelength
launch_align_file = 1
launch_height = inf
launch_tilt = 1
launch_type = LAUNCH_PLANEWAVE
launch_width = inf
sim_tool = ST_FULLWAVE
structure = STRUCT_FIBER
'''.format(**extra_config)

config['vars'] = rs.selfref_def_parser(config_text)

pillars = []
for num_pillars in range(40):
    pillar_x = np.random.uniform(-extra_config['ApertureRadius'], extra_config['ApertureRadius'])
    pillar_y = np.random.uniform(-extra_config['ApertureRadius'], extra_config['ApertureRadius'])
    pillar_diameter = np.random.uniform(0.5, 1.5)
    pillars.append([pillar_x, pillar_y, pillar_diameter])
config['segments'] = [
    substrate_segment
    ]
for idx, pillar in enumerate(pillars):
    pillar_x, pillar_y, pillar_diameter = pillar
    pillar_text = pillar_template.format(
        pillar_x = pillar_x, pillar_y = pillar_y, pillar_diameter = pillar_diameter)
    config['segments'].append(pillar_text)
for idx, segment in enumerate(config['segments']):
    config['segments'][idx] = rs.selfref_def_parser(segment,  config['vars'])

config['monitors'] = []
config['launch_fields'] = [rs.selfref_def_parser('''
	launch_pathway = 0
	launch_type = LAUNCH_PLANEWAVE
	launch_tilt = 1
	launch_align_file = 1
    ''', config['vars'])
	]
metal = rs.PhotoCircuit(config)
metal.save_to_file()


### ###Posts in square lattice

In [156]:
lmax = 10
dx = 1
even = True
num_steps = int(2*lmax/dx)
if num_steps % 2 == 0:
    rsteps = int(num_steps/2)
    rspace = np.linspace(dx/2, dx/2 + dx*rsteps, rsteps+1)[:-1]
    lspace = -rspace[-1::-1]
    evenspace = np.concatenate((lspace, rspace))
else:
    rsteps = int((num_steps-1)/2)
    rspace = np.linspace(0, dx*rsteps, rsteps+1)[:-1]
    lspace = -rspace[-1:0:-1]
    evenspace = np.concatenate((lspace, rspace))


In [262]:
def intervalspace(lmin, lmax, dx, symm='even'):
    '''
This  function  returns  a  numpy  array  of evenly spaced points
between  lmin  and  lmax with a spacing of dx. If symm is 'even',
the  array  will  have  an  even  number  of points symmetrically
distributed  about  the midpoint without including it. If symm is
'odd', the array will have an odd number of points with the given
points  symmetrically  distributed  about the midpoint, including
the  midpoint  itself. Given this the given array may not contain
the endpoints lmin and lmax.

    Parameters
    ----------
    lmin (float) : the minimum value of the interval
    lmax (float) : the maximum value of the interval
    dx (float)   : the spacing between points
    symm (str)   : wheter to include the midpoint or not.

    Returns
    -------
    interspace (numpy array): The evenly spaced array of points
    '''
    l = (lmax - lmin)/2
    rsteps = int(l/dx)
    if symm == 'even':
        rspace = np.linspace(dx/2, dx/2 + dx*rsteps, rsteps+1)[:-1]
        lspace = -rspace[-1::-1]
        interspace = np.concatenate((lspace, rspace))
    else:
        rspace = np.linspace(0, dx*rsteps, rsteps+1)
        lspace = -rspace[-1:0:-1]
        interspace = np.concatenate((lspace, rspace))
    interspace =  (lmin+lmax)/2 + interspace
    return interspace



In [263]:
intervalspace(-1,1,0.2,'odd')

array([-1. , -0.8, -0.6, -0.4, -0.2,  0. ,  0.2,  0.4,  0.6,  0.8,  1. ])

In [264]:
from itertools import product

In [303]:
apertureRadius = 10
period = 0.75
x = intervalspace(-apertureRadius, apertureRadius, period, 'odd')
y = intervalspace(-apertureRadius, apertureRadius, period, 'odd')
xgrid, ygrid = np.meshgrid(x,y)
radii = np.mod(10 - np.sqrt(xgrid**2 + ygrid**2 + 10**2) , period/2)


In [304]:
substrate_segment = '''
structure = STRUCT_CHANNEL
extended = 1
begin.x = 0
begin.z = -(2*Offset)
begin.height = 2*ApertureRadius
begin.width = 2*ApertureRadius
begin.delta = 2*SubstrateIndex - background_index
end.x = 0 rel begin segment 1
end.y = 0 rel begin segment 1
end.z = 0
end.height = 2*ApertureRadius
end.width = 2*ApertureRadius
end.delta = 2*SubstrateIndex-background_index
'''
pillar_template = '''
extended = 1
position_taper = TAPER_LINEAR
begin.x = {pillar_x}
begin.y = {pillar_y}
begin.z = 0
begin.width = {pillar_diameter}
begin.height = {pillar_diameter}
begin.delta = PillarIndex-background_index
end.x = {pillar_x}
end.y = {pillar_y}
end.z = PillarHeight
end.width = {pillar_diameter}
end.height = {pillar_diameter}
end.delta = PillarIndex-background_index
'''

In [305]:
extra_config = {
    'ApertureRadius' : 10,
    'PillarHeight': 1.2,
    'background_index': 1,
    'PillarIndex': 1.5,
    'free_space_wavelength': 0.600,
    'SubstrateIndex': 1.5}

config = {'vars'   : {}, 
        'segments' : [],
        'monitors' : [],
        'launch_fields' : {}
        }

config_text = '''
filename = metaopt.ind
ApertureRadius = {ApertureRadius}
Offset = 2*lambda
PillarHeight = {PillarHeight}
PillarIndex = {PillarIndex}
SubstrateIndex = {SubstrateIndex}
Xmax = {ApertureRadius}
Xmin = -{ApertureRadius}
Ymax = {ApertureRadius}
Ymin = -{ApertureRadius}
Zmax = {PillarHeight} + Offset
Zmin = -(2*Offset)
alpha = 0
background_index = {background_index}
boundary_max = Xmax
boundary_max_y = Ymax
boundary_min = Xmin
boundary_min_y = Ymin
cad_aspectratio = 1
delta = index-background_index
dimension = 3
domain_max = Zmax
domain_min = Zmin
eim = 0
fdtd_display_res_auto = DISPLAY_RES_AUTO
fdtd_monitor_time = lambda/4
fdtd_monitor_time_auto = MONITOR_TIME_AUTO
fdtd_pml_cells_enable = 1
fdtd_stop_auto = 1
fdtd_stop_time = 87
fdtd_stop_time_auto = 1
fdtd_time_step = 0.005681818182
fdtd_time_step_auto = 1
fdtd_update_time = 9*lambda/4
fdtd_update_time_auto = DISPLAY_TIME_AUTO
free_space_wavelength = {free_space_wavelength}
width = 1
height = width
index = 1
k0 = (2*pi)/free_space_wavelength
lambda = free_space_wavelength
launch_align_file = 1
launch_height = inf
launch_tilt = 1
launch_type = LAUNCH_PLANEWAVE
launch_width = inf
sim_tool = ST_FULLWAVE
structure = STRUCT_FIBER
'''.format(**extra_config)

config['vars'] = rs.selfref_def_parser(config_text)

pillars = []
for x,y,r in zip(np.ndarray.flatten(xgrid), np.ndarray.flatten(ygrid), np.ndarray.flatten(radii)):
    if x**2 + y**2 > extra_config['ApertureRadius']**2:
        continue
    pillar_x = x
    pillar_y = y
    pillar_diameter = r
    pillars.append([pillar_x, pillar_y, pillar_diameter])
config['segments'] = [
    substrate_segment
    ]
for idx, pillar in enumerate(pillars):
    pillar_x, pillar_y, pillar_diameter = pillar
    pillar_text = pillar_template.format(
        pillar_x = pillar_x, pillar_y = pillar_y, pillar_diameter = pillar_diameter)
    config['segments'].append(pillar_text)
for idx, segment in enumerate(config['segments']):
    config['segments'][idx] = rs.selfref_def_parser(segment,  config['vars'])

config['monitors'] = []
config['launch_fields'] = [rs.selfref_def_parser('''
	launch_pathway = 0
	launch_type = LAUNCH_PLANEWAVE
	launch_tilt = 1
	launch_align_file = 1
    ''', config['vars'])
	]
metal = rs.PhotoCircuit(config)
metal.save_to_file()
